In [3]:
from tkinter import *
import socket
import sys
import os
import logging
from threading import *
from math import floor
from time import time,sleep
from collections import deque
import socket
import pickle
from random import randint
SEQ_NUM_BIT_WIDTH = 3
MAX_SEQ_NUM = (2 ** SEQ_NUM_BIT_WIDTH)-1
GBN_WINDOW_SIZE = MAX_SEQ_NUM

class Packet:
    TYPE_DATA_,ACK_TYPE,NACK_TYPE = range(3)
    def __init__(self ,seq_num,data='', packtype=TYPE_DATA_):
        self.seq_num = seq_num
        self.data = data
        self.packtype = packtype
        self.corrupt = 0
    def is_corrupt(self):
        if not self.corrupt:
            self.corrupt = randint(1,10)
        return self.corrupt < 3
    def __str__(self):
        if self.packtype == Packet.TYPE_DATA_:
            return 'Packet(Seq_Num=%d, Data=%s) ' % (self.seq_num, str(self.data))
        elif self.packtype == Packet.NACK_TYPE:
            return 'Packet(NACK_Num=%d)' % self.seq_num
        elif self.packtype == Packet.ACK_TYPE:
            return 'Packet(ACK_Num=%d)' % self.seq_num

def SendPacket(sock, pack):
        if pack is None or (sock is None or type(sock) != socket.socket):
            return
        pack_raw_bytes = pickle.dumps(pack)
        dsize = len(pack_raw_bytes)
        sock.sendall(dsize.to_bytes(4, byteorder='big'))
        sock.sendall(pack_raw_bytes)
        print("Packet sent in SendPacket: ",pack)
        return True        

def read_k_bytes(sock, rem=0): 
        ret = b''
        while rem > 0:
            d = sock.recv(rem) 
            if len(d)<=0:
                k=0
                return k.to_bytes(4, byteorder='big')
            ret += d 
            rem -= len(d)
        return ret

def rec_pkt_nblock(sock):
        sock.setblocking(False)
        try:
            print("rec_pkt_nblock called")
            size = int.from_bytes(read_k_bytes(sock,4), 'big') 
            sock.setblocking(True)
            return pickle.loads(read_k_bytes(sock, size))   
        except BlockingIOError as be:
            return None
        finally:
            sock.setblocking(True)

sock=None
pbuffer=deque([],maxlen=GBN_WINDOW_SIZE) 
Send_f,Send_n=0,0 
message,msg_length='',0
client=None
ACK_WAIT_TIME=5000
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s.%(msecs)-03d : %(message)s',datefmt='%H:%M:%S')
class b_timer:
    def __init__(self):
        self.start_time=None
    def start(self,interval):
        print("Starting time")
        self.start_time=b_timer.current_time_in_millisec()
        self.interval=interval
    def has_timeout_occured(self):
        cur_time=b_timer.current_time_in_millisec()
        return (cur_time-self.start_time)>self.interval
    def is_running(self):
        return self.start_time!=None
    def stop(self):
        self.start_time=None
        self.interval=None
    def restart(self,interval):
        self.start(interval)
    @staticmethod
    def current_time_in_millisec():
        return int(floor(time()*1000))
    def outstanding_frames():
        return len(pbuffer) #upto date ack no
    def is_valid_ackno(ack_no):
        if b_timer.outstanding_frames()<=0:
            return False
        t=(Send_f+1)%(MAX_SEQ_NUM+1)
        while t!=Send_n:
            if t==ack_no :
                return True
            t=(t+1)%(MAX_SEQ_NUM+1)
        return ack_no==Send_n
def main():
        global pbuffer,client,Send_n,Send_f
        timer = b_timer()
        next_msg_index = 0 
        while 1:
            while b_timer.outstanding_frames()<GBN_WINDOW_SIZE and next_msg_index < msg_length:
                pack=Packet(Send_n,data=message[next_msg_index])
                SendPacket(client,pack)
                window.update()
                logging.info('[SEND] : Sending %s. '%pack)
                str1 = "[SEND] : Sending " + str(pack)+'\n'
                window.update()

                text.insert(END,str1)
                pbuffer.append(pack)
                Send_n=(Send_n+1)%(MAX_SEQ_NUM+1)
                print("Send_n: ",Send_n)
                if not timer.is_running():
                    #Pipelining
                    timer.start(ACK_WAIT_TIME)
                next_msg_index+=1
            sleep(.7)
            window.update()
            resp=rec_pkt_nblock(client)
            print("The received Packet is :",resp)
            str2 = "[RECV] : Recieved " + str(resp)+"\n"
            window.update()
            text.insert(END,str2)
            if resp is not None and not resp.is_corrupt():
                if not b_timer.is_valid_ackno(resp.seq_num):
                    logging.info('[NACK] :Invalid ACK %s.'%resp)
                    str3 = "[NACK] : Recieved " + str(resp)+'\n'
                    text.insert(END,str3)

                else:
                    temp=[] 
                    while len(pbuffer)>0 and pbuffer[0].seq_num!=resp.seq_num:
                        #sliding the window
                        temp.append(str(pbuffer.popleft().seq_num)) 
                        Send_f=(Send_f+1)%(MAX_SEQ_NUM+1) 
                        print("Send_f :",Send_f)
                        logging.info(('[RECEIVE] : ACK recieved %s.'+'Packets(%s) are acknowledged .')%(resp,','.join(temp)))
                        str4="[RECEIVE] : ACK Recieved for :"+str(resp)+" Packets Acknowledged "+ str(temp)
 
            sleep(.8)
            if timer.has_timeout_occured():
                for p in pbuffer:
                    logging.info('[TIMEOUT] : Resending %s.'%p)
                    str5="[TIMEOUT] : Resending " + str(p) + "\n"
                    window.update()
                    text.insert(END, str5)
                    SendPacket(client,p)
                timer.start(ACK_WAIT_TIME)
            if b_timer.outstanding_frames()==0 and next_msg_index>=msg_length:
                logging.info('Transfer Completed')
                window.update()
                text.insert(END, "\nTransfer Completed")
                break
            sleep(1)
def submit():
            global message,msg_length,client
            message = var.get()
            print(message)
            msg_length=len(message)
            print("Started From Server")
            text.insert(END, "Started From Server \n")
            sock=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            port=7917
            ip=socket.gethostname()
            sock.bind(('',port))
            sock.listen(5)
            client,_addr=sock.accept()
            print("Connection Accepted")
            window.update()
            main()
            print("Connection is closing ")
            text.insert(END, "\nConnection is closing ")
            sock.close()
            client.close()
if __name__=='__main__':
                window = Tk()
                window.geometry("900x557")
                Label(window,font=("Times New Roman",12),borderwidth=2,relief="solid",text= "SENDER").grid(row = 0, column = 0)
                Label(window,borderwidth=1,relief="solid",text= "ENTER A WORD ").grid(row = 1, column = 1)
                var = StringVar()
                window['background']='light grey'
                Entry(window, textvariable = var, width= 50,borderwidth = 5).grid(row = 2, column = 1) 
                text = Text(window,borderwidth=1,relief="solid")
                text['background']='light grey' 
                text.grid(row = 3, column = 0)
                text.configure(height=30)
                button = Button(window,text="SUBMIT",command=submit)
                button.grid(row = 2, column = 0)
                window.mainloop()
                






network
Started From Server


15:19:39.797 : [SEND] : Sending Packet(Seq_Num=0, Data=n) . 
15:19:39.814 : [SEND] : Sending Packet(Seq_Num=1, Data=e) . 
15:19:39.821 : [SEND] : Sending Packet(Seq_Num=2, Data=t) . 
15:19:39.826 : [SEND] : Sending Packet(Seq_Num=3, Data=w) . 
15:19:39.832 : [SEND] : Sending Packet(Seq_Num=4, Data=o) . 
15:19:39.836 : [SEND] : Sending Packet(Seq_Num=5, Data=r) . 
15:19:39.842 : [SEND] : Sending Packet(Seq_Num=6, Data=k) . 


Connection Accepted
Packet sent in SendPacket:  Packet(Seq_Num=0, Data=n) 
Send_n:  1
Starting time
Packet sent in SendPacket:  Packet(Seq_Num=1, Data=e) 
Send_n:  2
Packet sent in SendPacket:  Packet(Seq_Num=2, Data=t) 
Send_n:  3
Packet sent in SendPacket:  Packet(Seq_Num=3, Data=w) 
Send_n:  4
Packet sent in SendPacket:  Packet(Seq_Num=4, Data=o) 
Send_n:  5
Packet sent in SendPacket:  Packet(Seq_Num=5, Data=r) 
Send_n:  6
Packet sent in SendPacket:  Packet(Seq_Num=6, Data=k) 
Send_n:  7


15:19:40.567 : [RECEIVE] : ACK recieved Packet(ACK_Num=2).Packets(0) are acknowledged .
15:19:40.569 : [RECEIVE] : ACK recieved Packet(ACK_Num=2).Packets(0,1) are acknowledged .


rec_pkt_nblock called
The received Packet is : Packet(ACK_Num=2)
Send_f : 1
Send_f : 2


...

Packet sent in SendPacket:  Packet(Seq_Num=3, Data=w) 
Packet sent in SendPacket:  Packet(Seq_Num=4, Data=o) 
Packet sent in SendPacket:  Packet(Seq_Num=5, Data=r) 
Packet sent in SendPacket:  Packet(Seq_Num=6, Data=k) 
Starting time


15:19:58.345 : [RECEIVE] : ACK recieved Packet(ACK_Num=7).Packets(3) are acknowledged .
15:19:58.347 : [RECEIVE] : ACK recieved Packet(ACK_Num=7).Packets(3,4) are acknowledged .
15:19:58.350 : [RECEIVE] : ACK recieved Packet(ACK_Num=7).Packets(3,4,5) are acknowledged .
15:19:58.352 : [RECEIVE] : ACK recieved Packet(ACK_Num=7).Packets(3,4,5,6) are acknowledged .


rec_pkt_nblock called
The received Packet is : Packet(ACK_Num=7)
Send_f : 4
Send_f : 5
Send_f : 6
Send_f : 7


15:19:59.166 : Transfer Completed


Connection is closing 
